In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bfs
from IPython.display import HTML
import pandas as pd
import numpy as np
import time
import random
import os.path
import os
from Crypto.Cipher import AES
import binascii
import json

# Get all acode (runner unique code) in a run event

In [ ]:
HTML('<iframe src=https://services.datasport.com/2015/lauf/transviamala/alfaw.htm width=1000 height=350></iframe>')

In [ ]:
def get_all_acode(url):
    alpha_page = rq.get(url)
    alpha_page_soup = bfs(alpha_page.text, 'html5lib')

    runners = alpha_page_soup.body.findAll('span', attrs={'class': 'myds'})
    
    return {runner['acode']: str.strip(runner.text) for runner in runners}

def get_all_acode_from_run_event(url, file=False):
    
    if file and os.path.isfile(file):
        print('Read acode from file: ' + file)
        return pd.read_csv(file, index_col='acode').to_dict()['name']
    
    data = []
    page = rq.get(url)
    soup = bfs(page.text, 'html5lib')
    table_links = soup.select('font > a[href*=ALF]')
    all_acode = {}
    if url[-1] != '/':
        url += '/'
    for idx, link in enumerate(table_links):
        full_link = url + link['href']
        
        #print(str(idx+1) + '/' + str(len(table_links)) + ' - Processing ' + full_link)

        all_acode = {**all_acode, **get_all_acode(full_link)}
    if file:
        print('Write acode in file: ' + file)
        pd.Series(all_acode).to_csv(file, header=['name'], index_label='acode')
        
    return all_acode

In [ ]:
#acode_list = get_all_acode_from_run_event('https://services.datasport.com/2009/diverse/trophy/')

In [ ]:
#list(acode_list.items())[0:10]

# Get run events

In [ ]:
HTML('<iframe src=https://www.datasport.com/en/Calendar/ width=1000 height=350></iframe>')

## Get all params

In [ ]:
def get_all_params_calendar():
    calendar_page = rq.get('https://www.datasport.com/en/Calendar/')
    calendar_soup = bfs(calendar_page.text, 'html5lib')

    selector_table = calendar_soup.find('table', attrs={'id': 'ds-calendar-header'})

    available_params = {}
    for selector in selector_table.findAll('select'):
        available_params[selector['name']] = {option.text: option['value'] for option in selector.findAll('option')}

    return available_params

In [ ]:
#calendar_available_params = get_all_params_calendar()
#calendar_available_params

## Get all run events url in a calendar

In [ ]:
def get_run_events_url(year=2009, month=1, country='CCH', sport='Running'):

    calendar_params = {
        'dr': '',
        'lastQuery': 'D147BC896417D2D2B96FA1AADD893731',
        'eventsearch': '',
        'eventservice': 'all',
        'start': 1,
        'etyp': sport,
        'eventlocation': country,
        'eventmonth': month,
        'eventyear': year,
    }

    calendar_page = rq.post('https://www.datasport.com/en/Calendar/', data = calendar_params)
    calendar_soup = bfs(calendar_page.text, 'html5lib')

    table = calendar_soup.find('table', attrs={'id': 'ds-calendar-body'})

    all_event_url = {}
    for row in table.findAll('tr'):
        columns = row.findAll('td')
        if len(columns) >= 4:
            url = columns[4].find('a')
            if url:
                all_event_url[url['href']] = {
                    'year': year,
                    'month': month,
                    'country': country,
                    'sport': sport,
                    'full_date': str.strip(columns[0].text),
                    'name': str.strip(columns[1].text),
                }

    return all_event_url

In [ ]:
#get_run_events_url()

## Get all run event urls

In [ ]:
def get_all_run_events():
    run_events = {}
    calendar_available_params = get_all_params_calendar()
    for year_text, year_value in calendar_available_params['eventyear'].items():
        if year_value != 'all':
            for month_text, month_value in calendar_available_params['eventmonth'].items():
                if month_value != 'all':
                    print('Processing: Year ' + year_value + ' / Month ' + month_value)
                    run_events = {**run_events,  **get_run_events_url(year_value, month_value)}
                    time.sleep(random.uniform(0.5, 2))
    return run_events

In [ ]:
#run_events = get_all_run_events()

In [ ]:
#run_events_df = pd.DataFrame(run_events).T
#run_events_df.index.name = 'url'

In [ ]:
#run_events_df.to_csv('Data/run_events.csv')

In [ ]:
!head Data/run_events.csv

In [ ]:
run_events_df = pd.read_csv('Data/run_events.csv')
run_events_df.head()

## Get all acode in all runs from 2009

In [ ]:
#run_events_df_2009_2015 = run_events_df[[year >= 2009 and year < 2016 for year in run_events_df['year']]]
#run_events_df_2009_2015.size

In [ ]:
#run_events_df_2009_2015.to_csv('Data/run_events_2009_2015.csv', index_label='acode_index')
!head Data/run_events_2009_2015.csv

In [ ]:
run_events_df_2009_2015 = pd.read_csv('Data/run_events_2009_2015.csv', index_col='acode_index')

In [ ]:
def write_all_acode_from_list_run_event(run_events, path='Data/acode_2009_2015/'):
    for run_event in run_events.itertuples():
        print('Processing: Run "' + run_event.name + '" / Date ' + run_event.full_date + ' / Url ' + run_event.url)
        get_all_acode_from_run_event(run_event.url, path + str(run_event.Index) + '.csv')

In [ ]:
PATH_TO_DATA = './Data/acode_2009_2015/'

In [ ]:
#all_acode_2009_2015 = get_all_acode_from_list_run_event(run_events_df_2009_2015, PATH_TO_DATA)

In [ ]:
# definir une méthode

# get all data and remove the double
file_list = os.listdir(PATH_TO_DATA)
result_dataframe = pd.read_csv(PATH_TO_DATA + '1000.csv')
for file in file_list:
    pd.concat([result_dataframe, pd.read_csv(PATH_TO_DATA+file)], axis=0)

In [ ]:
result_dataframe.head()

In [ ]:
len(result_dataframe)

In [ ]:
FILE_NAME = 'unique_acode_2009_2015.csv'
if(len(result_dataframe['acode'].unique()) == len(result_dataframe)):
    result_dataframe.to_csv(PATH_TO_DATA+FILE_NAME)

# Get runner information

In [ ]:
KEY = '0187fcdf7ac3d90d68334afa03b87efe' # Decode Base64 string + Hex encoder
IV = '33a39433e4dde7c4ddb9d4502b8905d4' # Decode Base64 string + Hex encoder
KEY_BYTES = binascii.a2b_hex(KEY)
IV_BYTES = binascii.a2b_hex(IV)
CIPHER = AES.new(KEY_BYTES, AES.MODE_CBC, IV_BYTES, segment_size=128)

In [ ]:
def unpad(string):
    '''
    Remove the PKCS#7 padding from a text string
    '''

    length = len(string)
    pad_size = string[-1]
    return string[:length - pad_size]


def decrypt_data(cipher, encrypted):
    '''
    Decrypt data given key and iv
    '''

    decrypted = cipher.decrypt(binascii.a2b_base64(encrypted).rstrip())
    return unpad(decrypted)


def parse_html_runner_table(table, base_url='https://www.datasport.com'):

    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    
    
    url_information_runner = []
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        
        columns = row.find_all('td')
        for index, column in enumerate(columns):
            df.iat[row_marker,column_marker] = column.get_text()
            
            # add the url to get informations about the runners.
            if(index == 1):
                url_information_runner.append(base_url + column.find('a')['href'])
                
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    #print(url_information_runner)
    # add columns url
    df['url_run_event'] = pd.Series(url_information_runner)
    return df


def get_runner_information(acode_file='./Data/acode_2009_2015/unique_acode_2009_2015.csv', base_url='https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode='):

    data_acode = pd.read_csv(acode_file)

    ATTRIBUTE_RUN = 'url_run_event'
    for acode in data_acode['acode']:
        
        url = base_url + acode
        
        # We retrieve filters and store cookies for further calls to server
        filters_page = rq.get(url)
        cookies = filters_page.cookies.get_dict()
        page = bfs(filters_page.text, 'html.parser')

        # We get the birthyear
        small_p = page.findAll('p', { 'class': 'small' })
        birth_year = small_p[1].text.split(" ")[1:][0]

        # We get information on the table
        table_run_event = page.find('table', { 'id': 'timeTable' })
        information_runner = parse_html_runner_table(table_run_event)

        for index, row in information_runner.iterrows():
            url_ajax = row[ATTRIBUTE_RUN]
            ajax_response = rq.get(url_ajax, cookies=cookies)
            decrypted_response = decrypt_data(cipher=CIPHER, encrypted=ajax_response.text)
            print(url_ajax)
            print(decrypted_response)  
            break

        break

In [ ]:
data_acode = pd.read_csv('./Data/acode_2009_2015/unique_acode_2009_2015.csv')
data_acode.head()

In [ ]:
get_runner_information()